### Importing libraries

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical 

### Loading Dataset (sonnets of shakespear)

In [4]:
text=(open("./sonnets.txt").read())
text=text.lower()

### Chart to vector map

In [5]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

### Preparing Data

In [6]:
x = []
y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    x.append([char_to_n[char] for char in sequence])
    y.append(char_to_n[label])

In [7]:
len(x)

119649

### Data reformation

In [9]:
x_modified = np.reshape(x, (len(x), seq_length, 1))
x_modified = x_modified / float(len(characters))
y_modified = to_categorical(y)

In [10]:
x_modified.shape

(119649, 100, 1)

In [11]:
y_modified.shape

(119649, 57)

### Model

In [12]:
model = Sequential()
model.add(LSTM(700, input_shape=(x_modified.shape[1], x_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 700)          1965600   
_________________________________________________________________
dropout (Dropout)            (None, 100, 700)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 700)          3922800   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 700)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 700)               3922800   
_________________________________________________________________
dropout_2 (Dropout)          (None, 700)               0         
_________________________________________________________________
dense (Dense)                (None, 57)                39957     
Total para

In [14]:
model.fit(x_modified, y_modified, epochs=100, batch_size=50)

model.save('./poemModel.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
   300/119649 [..............................] - ETA: 6:34:48 - loss: 3.8403

KeyboardInterrupt: 

### Generating Text

In [ ]:
string_mapped = x[99]
full_string = [n_to_char[value] for value in string_mapped]

for i in range(400):
    xtest = np.reshape(string_mapped,(1,len(string_mapped), 1))
    xtest = xtest / float(len(characters))

    pred_index = np.argmax(model.predict(xtest, verbose=0))
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

### Output

In [ ]:
txt=""
for char in full_string:
    txt = txt+char
txt